In [1]:
import os
import sys
import numpy as np
import pandas as pd
import warnings
import logging.config
import matplotlib.pyplot as plt

os.environ["OMP_NUM_THREADS"] = "4"
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["MKL_NUM_THREADS"] = "6"
os.environ["VECLIB_MAXIMUM_THREADS"] = "4"
os.environ["NUMEXPR_NUM_THREADS"] = "6"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
warnings.simplefilter('ignore')

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
sys.path = sys.path[1:]
sys.path.append(os.path.abspath(os.pardir))
#sys.path.append(os.path.join(os.path.abspath(os.pardir), 'odometry'))
sys.path.append(os.path.join(os.path.abspath(os.pardir), 'submodules/tfoptflow/tfoptflow'))
sys.path.append(os.path.join(os.path.abspath(os.pardir), 'submodules/tf_models/research/struct2depth'))

In [3]:
sys.path.append(os.path.join(os.path.abspath(os.pardir), 'odometry'))

In [4]:
from odometry.preprocessing.parsers.discoman_parser import DISCOMANCSVParser

In [5]:
sequence_directory = 'discomancsv'
csv_path = '/dbstore/datasets/Odometry_team/discoman_v10_unzip/000005/camera_gt.csv'
height, width = 90, 160

In [6]:
discoman_csv_parser = DISCOMANCSVParser(sequence_directory, csv_path=csv_path)

In [7]:
discoman_csv_parser

In [9]:
a = discoman_csv_parser.run()

In [10]:
a.path_to_rgb[0]

'/dbstore/datasets/Odometry_team/discoman_v10_unzip/000005/000000_raycast.jpg'

In [8]:
from odometry.preprocessing.parsers.elementwise_parser import ElementwiseParser

In [10]:
import pandas as pd

In [13]:
from odometry.preprocessing.prepare_trajectory import prepare_trajectory

In [14]:
from odometry.preprocessing.estimators import *

In [15]:
quaternion2euler_estimator = Quaternion2EulerEstimator(input_col=['q_w', 'q_x', 'q_y', 'q_z'],
                                                       output_col=['euler_x', 'euler_y', 'euler_z'])
quaternion2euler_estimator

Quaternion2EulerEstimator(input_col=['q_w', 'q_x', 'q_y', 'q_z'], output_col=['euler_x', 'euler_y', 'euler_z'])

In [16]:
single_frame_estimators = [quaternion2euler_estimator]

In [17]:
input_col = ['euler_x', 'euler_y', 'euler_z', 't_x', 't_y', 't_z']
input_col.extend([col + '_next' for col in input_col])
output_col = input_col
global2relative_estimator = Global2RelativeEstimator(input_col=input_col, output_col=output_col)
global2relative_estimator

Global2RelativeEstimator(input_col=['euler_x', 'euler_y', 'euler_z', 't_x', 't_y', 't_z', 'euler_x_next', 'euler_y_next', 'euler_z_next', 't_x_next', 't_y_next', 't_z_next'], output_col=['euler_x', 'euler_y', 'euler_z', 't_x', 't_y', 't_z', 'euler_x_next', 'euler_y_next', 'euler_z_next', 't_x_next', 't_y_next', 't_z_next'])

In [19]:
optical_flow_checkpoint = '/Vol0/user/f.konokhov/tfoptflow/tfoptflow/tmp/pwcnet.ckpt-84000'
#optical_flow_checkpoint = os.path.abspath(../weights/pwcnet.ckpt-595000')  # official weights
pwcnet_estimator = PWCNetEstimator(input_col=['path_to_rgb', 'path_to_rgb_next'],
                                   output_col='path_to_optical_flow',
                                   sub_dir='optical_flow',
                                   checkpoint=optical_flow_checkpoint,
                                   height=height,
                                   width=width)
pwcnet_estimator

Building model...
... model built.
Loading model checkpoint /Vol0/user/f.konokhov/tfoptflow/tfoptflow/tmp/pwcnet.ckpt-84000 for eval or testing...

INFO:tensorflow:Restoring parameters from /Vol0/user/f.konokhov/tfoptflow/tfoptflow/tmp/pwcnet.ckpt-84000


I0619 16:29:01.080954 47257034980992 tf_logging.py:115] Restoring parameters from /Vol0/user/f.konokhov/tfoptflow/tfoptflow/tmp/pwcnet.ckpt-84000


... model loaded


PWCNetEstimator(dir=optical_flow, input_col=['path_to_rgb', 'path_to_rgb_next'], output_col=path_to_optical_flow, checkpoint=/Vol0/user/f.konokhov/tfoptflow/tfoptflow/tmp/pwcnet.ckpt-84000)

In [20]:
pair_frames_estimators = [global2relative_estimator, pwcnet_estimator]

In [21]:
import glob

In [22]:
allfolders = glob.glob('/dbstore/datasets/Odometry_team/discoman_v10_unzip/*/')
allfolders.sort()

In [ ]:
for folder in allfolders:
    folder_name = os.path.basename(os.path.normpath(folder))
    sequence_directory = '/dbstore/datasets/Odometry_team/discoman_v10_full/{}'.format(folder_name)
    csv_path = '{}camera_gt.csv'.format(folder)
    discoman_csv_parser = DISCOMANCSVParser(sequence_directory, csv_path=csv_path)
    prepare_trajectory(sequence_directory, 
                   parser=discoman_csv_parser, 
                   single_frame_estimators=single_frame_estimators,
                   pair_frames_estimators=pair_frames_estimators,
                   stride=1).to_csv(os.path.join(sequence_directory, 'df.csv'))

    

PWCNet:  37%|███▋      | 714/1910 [00:40<01:06, 18.04it/s]